In [ ]:
# Cell 1: Imports and Setup
import cv2
import mediapipe as mp
import pygame
import math
import random
import time
import os
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt

# --- 1. SETTINGS ---
pygame.init()
WIDTH, HEIGHT = 1280, 720
# Note: This will launch a separate popup window when run
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("AR Chicken - Notebook Edition")

# Colors
RED = (255, 50, 50)
GREEN = (50, 255, 50)
BLUE = (50, 150, 255)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
ORANGE = (255, 165, 0)
PINK = (255, 105, 180)
EGG_WHITE = (250, 245, 220)
BASKET_COLOR = (139, 69, 19)
DARK_GREY = (50, 50, 50) # Fallback color for Pan

# Game Balance
MAX_HUNGER = 100
MAX_GROWTH = 100
HUNGER_DECAY = 0.02
GROWTH_RATE = 0.05
BASE_MOVE_SPEED = 2
ZOOMIE_SPEED = 10
MAX_BASKET_CAPACITY = 3

# Pomodoro
STUDY_MINUTES = 15
BREAK_MINUTES = 5
GAME_STATE = "STUDY"

# AI Models

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
phone_model = YOLO("yolov8n.pt")



In [ ]:
# Cell 2: Game Classes

def load_image(filename, size=None):
    try:
        if os.path.exists(filename):
            img = pygame.image.load(filename).convert_alpha()
            if size: img = pygame.transform.smoothscale(img, size)
            return img
    except: pass
    return None


CHICKEN_IMG = load_image("CHICKEN173.png")
BASKET_IMG = load_image("EGGBASKET173.png", (120, 100))
HAY_IMG = load_image("HAY173.png", (100, 100))
FEED_IMG = load_image("FEED173.png", (80, 80))
PAN_IMG = None
OMELETTE_IMG = None

class HeartParticle:
    def __init__(self, x, y):
        self.x = x + random.randint(-20, 20)
        self.y = y - 20
        self.size = random.randint(10, 20)
        self.life = 255
        self.speed = random.uniform(1, 3)

    def update(self):
        self.y -= self.speed
        self.life -= 5

    def draw(self, surface):
        if self.life > 0:
            s = pygame.Surface((self.size, self.size), pygame.SRCALPHA)
            pygame.draw.circle(s, (255, 105, 180, self.life), (self.size//2, self.size//2), self.size//2)
            surface.blit(s, (self.x, self.y))

class Egg:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, 40, 50)
        self.color = EGG_WHITE
        self.is_held = False

    def draw(self, surface):
        pygame.draw.ellipse(surface, self.color, self.rect)
        pygame.draw.ellipse(surface, BLACK, self.rect, 4)

class Hay:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, 80, 80)
        self.color = YELLOW
        self.is_held = True

    def draw(self, surface):
        if FEED_IMG: surface.blit(FEED_IMG, self.rect)
        else:
            pygame.draw.rect(surface, self.color, self.rect)
            pygame.draw.rect(surface, BLACK, self.rect, 2)

class Chicken:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, 60, 60)
        self.color = RED
        self.is_held = False
        self.hunger = 80
        self.growth = 0
        self.adult_egg_laid = False
        self.target_x = x
        self.target_y = y
        self.move_timer = 0
        self.state = "idle"
        self.zoomie_timer = 0

    def update(self, eggs_list):
        if self.hunger > 0:
            self.hunger -= HUNGER_DECAY
            if self.hunger > 50: self.growth += GROWTH_RATE

        self.hunger = max(0, min(self.hunger, MAX_HUNGER))
        self.growth = min(self.growth, MAX_GROWTH)

        if self.growth >= 100 and not self.adult_egg_laid:
            eggs_list.append(Egg(self.rect.centerx, self.rect.bottom + 10))
            self.adult_egg_laid = True

        if not self.is_held:
            if self.state == "panicked":
                self.rect.x += random.randint(-5, 5)
                self.rect.y += random.randint(-5, 5)
                return

            current_speed = BASE_MOVE_SPEED
            if self.zoomie_timer > 0:
                self.zoomie_timer -= 1
                current_speed = ZOOMIE_SPEED

            dist = math.hypot(self.target_x - self.rect.centerx, self.target_y - self.rect.centery)
            if dist < 10:
                self.move_timer += 1
                threshold = 10 if self.zoomie_timer > 0 else 100
                if self.move_timer > threshold:
                    self.move_timer = 0
                    self.target_x = random.randint(100, WIDTH - 100)
                    self.target_y = random.randint(100, HEIGHT - 100)
            else:
                dx = self.target_x - self.rect.centerx
                dy = self.target_y - self.rect.centery
                self.rect.centerx += int(dx / dist * current_speed)
                self.rect.centery += int(dy / dist * current_speed)

    def eat(self, particles_list):
        self.hunger += 20
        size_bonus = int(self.growth * 0.5)
        self.rect.width = 60 + size_bonus
        self.rect.height = 60 + size_bonus
        self.zoomie_timer = 120
        for _ in range(5):
            particles_list.append(HeartParticle(self.rect.centerx, self.rect.y))

    def draw(self, surface):
        if CHICKEN_IMG:
            scaled_chicken = pygame.transform.smoothscale(CHICKEN_IMG, (self.rect.width, self.rect.height))
            surface.blit(scaled_chicken, (self.rect.x, self.rect.y))
        else:
            pygame.draw.rect(surface, self.color, self.rect)
            pygame.draw.circle(surface, BLACK, (self.rect.right - 10, self.rect.y + 15), 3)

        bar_x = self.rect.x
        bar_y = self.rect.y - 20
        pygame.draw.rect(surface, BLACK, (bar_x, bar_y, 60, 5))
        pygame.draw.rect(surface, GREEN, (bar_x, bar_y, max(0, self.hunger / 100 * 60), 5))
        bar_y -= 8
        pygame.draw.rect(surface, BLACK, (bar_x, bar_y, 60, 5))
        pygame.draw.rect(surface, BLUE, (bar_x, bar_y, max(0, self.growth / 100 * 60), 5))

class CookingGame:
    def __init__(self):
        self.phase = "CHOP"
        self.progress = 0
        self.max_progress = 100
        self.prev_y = 0
        self.last_angle = 0
        self.center_pan = (WIDTH//2, HEIGHT//2)
        self.pan_rect = pygame.Rect(WIDTH//2 - 200, HEIGHT//2 - 200, 400, 400)

    def reset(self):
        self.phase = "CHOP"
        self.progress = 0
        self.prev_y = 0
        self.last_angle = 0

    def process_gestures(self, cx, cy):
        if self.phase == "CHOP":
            velocity = abs(cy - self.prev_y)
            if velocity > 15: self.progress += 2
            self.prev_y = cy
            if self.progress >= self.max_progress:
                self.phase = "STIR"
                self.progress = 0
                time.sleep(0.5)
        elif self.phase == "STIR":
            dx = cx - self.center_pan[0]
            dy = cy - self.center_pan[1]
            angle = math.degrees(math.atan2(dy, dx))
            diff = abs(angle - self.last_angle)
            if diff > 10 and diff < 100: self.progress += 3
            self.last_angle = angle
            if self.progress >= self.max_progress:
                self.phase = "DONE"

    def draw(self, surface):
        overlay = pygame.Surface((WIDTH, HEIGHT))
        overlay.set_alpha(200)
        overlay.fill(BLACK)
        surface.blit(overlay, (0,0))


        pygame.draw.circle(surface, DARK_GREY, self.center_pan, 200)
        pygame.draw.circle(surface, (30, 30, 30), self.center_pan, 180)

        if self.phase == "CHOP":
            txt = pygame.font.SysFont("arial", 36, bold=True).render("CHOP! (Move hand UP/DOWN)", True, WHITE)
            surface.blit(txt, (WIDTH//2 - 200, 100))
            pygame.draw.ellipse(surface, YELLOW, (WIDTH//2-50, HEIGHT//2-50, 100, 100))
        elif self.phase == "STIR":
            txt = pygame.font.SysFont("arial", 36, bold=True).render("STIR! (Spin hand around)", True, WHITE)
            surface.blit(txt, (WIDTH//2 - 150, 100))
            pygame.draw.circle(surface, YELLOW, self.center_pan, 100)
        elif self.phase == "DONE":
            txt = pygame.font.SysFont("arial", 36, bold=True).render("DELICIOUS! (Click to Eat)", True, GREEN)
            surface.blit(txt, (WIDTH//2 - 250, 100))
            # DRAW OMELETTE (Shape)
            pygame.draw.ellipse(surface, (255, 215, 0), (WIDTH//2-100, HEIGHT//2-80, 200, 160))

        if self.phase != "DONE":
            pygame.draw.rect(surface, WHITE, (WIDTH//2 - 150, HEIGHT - 100, 300, 30))
            pygame.draw.rect(surface, GREEN, (WIDTH//2 - 150, HEIGHT - 100, self.progress * 3, 30))

In [ ]:
# Cell 3: Main Execution Loop

def run_game():
    global GAME_STATE
    cap = cv2.VideoCapture(0)

    # Initialize Objects
    my_chicken = Chicken(WIDTH // 2, HEIGHT // 2)
    cooking_game = CookingGame()
    active_hay = None
    held_egg = None
    eggs_list = [Egg(200, 200), Egg(260, 200), Egg(320, 200)]
    eggs_collected = 0
    particles = []

    # Timers
    timer_active = False
    study_time_left = STUDY_MINUTES * 60
    break_time_left = BREAK_MINUTES * 60
    last_time_check = time.time()

    # UI Rects
    hay_pile_rect = pygame.Rect(WIDTH - 150, HEIGHT - 150, 100, 100)
    timer_button_rect = pygame.Rect(WIDTH - 150, 50, 100, 100)
    basket_rect = pygame.Rect(50, HEIGHT - 150, 120, 100)
    cook_button_rect = pygame.Rect(50, HEIGHT - 250, 200, 60)

    clock = pygame.time.Clock()
    running = True
    button_cooldown = 0
    frame_count = 0
    phone_detected = False

    font = pygame.font.SysFont("arial", 24, bold=True)
    large_font = pygame.font.SysFont("arial", 36, bold=True)

    def calculate_distance(p1, p2):
        return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

    def draw_ui_box(surface, rect, color, text):
        s = pygame.Surface((rect.width, rect.height))
        s.set_alpha(180)
        s.fill(color)
        surface.blit(s, (rect.x, rect.y))
        pygame.draw.rect(surface, BLACK, rect, 3)
        txt_surf = font.render(text, True, BLACK)
        txt_rect = txt_surf.get_rect(center=rect.center)
        surface.blit(txt_surf, txt_rect)

    while running:
        frame_count += 1
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                cap.release()
                pygame.quit()
                return

        success, image = cap.read()
        if not success: continue

        image = cv2.flip(image, 1)
        image_resized = cv2.resize(image, (WIDTH, HEIGHT))

        # YOLO Detection (Phone/Remote)
        if GAME_STATE == "STUDY" and frame_count % 30 == 0:
            yolo_results = phone_model(image_resized, verbose=False)
            phone_detected = False
            for r in yolo_results:
                boxes = r.boxes
                for box in boxes:
                    if (int(box.cls[0]) == 67 or int(box.cls[0]) == 65) and float(box.conf[0]) > 0.4:
                        phone_detected = True

        frame_rgb = cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB)
        frame_surface = pygame.surfarray.make_surface(frame_rgb.swapaxes(0, 1))
        screen.blit(frame_surface, (0, 0))

        results = hands.process(frame_rgb)
        cursor_x, cursor_y = -100, -100
        is_pinching = False

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                index_tip = hand_landmarks.landmark[8]
                thumb_tip = hand_landmarks.landmark[4]
                cx, cy = int(index_tip.x * WIDTH), int(index_tip.y * HEIGHT)
                if cursor_x == -100: cursor_x, cursor_y = cx, cy
                if calculate_distance(index_tip, thumb_tip) < 0.05:
                    is_pinching = True

        # --- LOGIC HANDLING ---
        if GAME_STATE == "STUDY":
            if phone_detected and timer_active:
                my_chicken.state = "panicked"
                pygame.draw.rect(screen, RED, (0,0,WIDTH,HEIGHT), 10)
            else:
                my_chicken.state = "idle"

            if timer_active and not phone_detected:
                if time.time() - last_time_check >= 1.0:
                    study_time_left -= 1
                    last_time_check = time.time()
                    if study_time_left <= 0:
                        timer_active = False
                        study_time_left = STUDY_MINUTES * 60
                        my_chicken.growth += 15
                        eggs_list.append(Egg(my_chicken.rect.centerx, my_chicken.rect.bottom))
            else:
                last_time_check = time.time()

            if button_cooldown > 0: button_cooldown -= 1
            my_chicken.update(eggs_list)

            if is_pinching:
                if button_cooldown == 0 and timer_button_rect.collidepoint(cursor_x, cursor_y):
                    timer_active = not timer_active
                    button_cooldown = 60

                if button_cooldown == 0 and eggs_collected >= MAX_BASKET_CAPACITY and cook_button_rect.collidepoint(cursor_x, cursor_y):
                    GAME_STATE = "COOKING"
                    eggs_collected = 0
                    cooking_game.reset()
                    break_time_left = BREAK_MINUTES * 60
                    timer_active = False
                    button_cooldown = 60

                if not my_chicken.is_held and active_hay is None and held_egg is None:
                    if my_chicken.rect.collidepoint(cursor_x, cursor_y): my_chicken.is_held = True
                    elif hay_pile_rect.collidepoint(cursor_x, cursor_y): active_hay = Hay(cursor_x, cursor_y)
                    else:
                        for egg in eggs_list:
                            if egg.rect.collidepoint(cursor_x, cursor_y):
                                egg.is_held = True
                                held_egg = egg
                                break

            if not is_pinching:
                my_chicken.is_held = False
                if active_hay:
                    if my_chicken.rect.colliderect(active_hay.rect):
                        my_chicken.eat(particles)
                    active_hay = None
                if held_egg:
                    held_egg.is_held = False
                    if basket_rect.colliderect(held_egg.rect):
                        if eggs_collected < MAX_BASKET_CAPACITY:
                            if held_egg in eggs_list:
                                eggs_list.remove(held_egg)
                                eggs_collected += 1
                    held_egg = None

            if my_chicken.is_held: my_chicken.rect.center = (cursor_x, cursor_y)
            if active_hay: active_hay.rect.center = (cursor_x, cursor_y)
            if held_egg: held_egg.rect.center = (cursor_x, cursor_y)

            # Drawing
            if HAY_IMG: screen.blit(HAY_IMG, hay_pile_rect)
            else: draw_ui_box(screen, hay_pile_rect, YELLOW, "Feed")

            btn_color = GREEN if timer_active else BLUE
            draw_ui_box(screen, timer_button_rect, btn_color, "PAUSE" if timer_active else "STUDY")

            if BASKET_IMG: screen.blit(BASKET_IMG, basket_rect)
            else: draw_ui_box(screen, basket_rect, BASKET_COLOR, "Basket")

            if eggs_collected >= MAX_BASKET_CAPACITY:
                draw_ui_box(screen, cook_button_rect, ORANGE, "START BREAK")

            cnt_txt = large_font.render(f"{eggs_collected}/{MAX_BASKET_CAPACITY}", True, WHITE)
            screen.blit(cnt_txt, (basket_rect.centerx - 20, basket_rect.top - 40))

            mins, secs = int(study_time_left // 60), int(study_time_left % 60)
            t_surf = pygame.font.SysFont("arial", 50, bold=True).render(f"{mins:02}:{secs:02}", True, WHITE)
            pygame.draw.rect(screen, BLACK, (WIDTH//2 - 70, 10, 140, 60))
            screen.blit(t_surf, (WIDTH//2 - 60, 15))

            for egg in eggs_list:
                if not egg.is_held: egg.draw(screen)
            my_chicken.draw(screen)
            for p in particles[:]:
                p.update()
                p.draw(screen)
                if p.life <= 0: particles.remove(p)
            if held_egg: held_egg.draw(screen)
            if active_hay: active_hay.draw(screen)

        elif GAME_STATE == "COOKING":
            if time.time() - last_time_check >= 1.0:
                break_time_left -= 1
                last_time_check = time.time()
                if break_time_left <= 0:
                    GAME_STATE = "STUDY"
                    my_chicken.eat(particles)
                    break_time_left = BREAK_MINUTES * 60

            if cursor_x > 0: cooking_game.process_gestures(cursor_x, cursor_y)
            cooking_game.draw(screen)

            mins, secs = int(break_time_left // 60), int(break_time_left % 60)
            break_txt = font.render(f"BREAK TIME: {mins:02}:{secs:02}", True, WHITE)
            pygame.draw.rect(screen, BLACK, (WIDTH//2 - 100, 20, 200, 40))
            screen.blit(break_txt, (WIDTH//2 - 80, 25))

            if cooking_game.phase == "DONE" and is_pinching and button_cooldown == 0:
                GAME_STATE = "STUDY"
                my_chicken.eat(particles)
                button_cooldown = 60
                break_time_left = BREAK_MINUTES * 60

        if cursor_x > 0:
            col = (0, 255, 0) if is_pinching else (0, 0, 255)
            pygame.draw.circle(screen, col, (cursor_x, cursor_y), 10)

        pygame.display.flip()
        clock.tick(60)

# Run the game
if __name__ == "__main__":
    run_game()